<a href="https://colab.research.google.com/github/Miekevi/masterproject/blob/main/LLMSciBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
DATADIR="NCBI-disease"
if test ! -d "$DATADIR";then
    echo "Creating $DATADIR dir"
    mkdir "$DATADIR"
    cd "$DATADIR"
    wget https://raw.githubusercontent.com/allenai/scibert/master/data/ner/NCBI-disease/dev.txt
    wget https://raw.githubusercontent.com/allenai/scibert/master/data/ner/NCBI-disease/test.txt
    wget https://raw.githubusercontent.com/allenai/scibert/master/data/ner/NCBI-disease/train.txt
fi

Creating NCBI-disease dir


--2023-10-15 22:48:37--  https://raw.githubusercontent.com/allenai/scibert/master/data/ner/NCBI-disease/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 320197 (313K) [text/plain]
Saving to: ‘dev.txt’

     0K .......... .......... .......... .......... .......... 15% 7.10M 0s
    50K .......... .......... .......... .......... .......... 31% 11.7M 0s
   100K .......... .......... .......... .......... .......... 47% 11.4M 0s
   150K .......... .......... .......... .......... .......... 63% 5.95M 0s
   200K .......... .......... .......... .......... .......... 79% 13.4M 0s
   250K .......... .......... .......... .......... .......... 95% 36.8M 0s
   300K .......... ..                                         100% 82.2M=0.03s

2023-10-15 22:48:37 (10

# New section

In [ ]:
import os
import math
import random
import csv
import sys

import sklearn
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
import statistics as stats

sys.path.append("../")
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model

def read_tsv(filename, quotechar=None):
    with open(filename, "r", encoding='utf-8') as f:
        return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

def flatten(l):
    return [item for sublist in l for item in sublist]

def read_CoNLL2003_format(filename, idx=3):
    """Read file in CoNLL-2003 shared task format"""

    # read file
    lines =  open(filename).read().strip()

    # find sentence-like boundaries
    lines = lines.split("\n\n")

     # split on newlines
    lines = [line.split("\n") for line in lines]

    # get tokens
    tokens = [[l.split()[0] for l in line] for line in lines]

    # get labels/tags
    labels = [[l.split()[idx] for l in line] for line in lines]

    #convert to df
    data= {'tokens': tokens, 'labels': labels}
    df=pd.DataFrame(data=data)
    return df

DATADIR = "NCBI-disease/"

def get_data(trainfile=DATADIR + "train.txt",
             devfile=DATADIR + "dev.txt",
             testfile=DATADIR + "test.txt"):

    train = read_CoNLL2003_format(trainfile, idx=3)
    dev = read_CoNLL2003_format(devfile, idx=3)

    # combine train and dev
    train = pd.concat([train, dev])
    print("Train and dev data: %d sentences, %d tokens"%(len(train),len(flatten(train.tokens))))

    test = read_CoNLL2003_format(testfile, idx=3)
    print("Test data: %d sentences, %d tokens"%(len(test),len(flatten(test.tokens))))

    return train, test

train, test = get_data()

X_train, y_train = train.tokens, train.labels
X_test, y_test = test.tokens, test.labels

print(len(train))

label_list = np.unique(flatten(y_train))
label_list = list(label_list)
print("\nNER tags:",label_list)

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 7.37 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Processing /content/bert-sklearn
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.9 MB/s eta 0:00:00
  Created wheel for bert-sklearn: filename=bert_sklearn-0.3.1-py3-none-any.whl size=54223 sha256=921f660876eb89cfffeb15cdab604473779cb3a444ca6f252e2f19f2b254b6cc
  Stored in directory: /root/.cache/pip/wheels/29/f7/03/c9d257b287e1a5ef7ee4300cbbcd70cfbb13e8b6759da24477
Successfully built bert-sklearn
Train and dev data: 6347 sentences, 159670 tokens
Test data: 940 sentences, 24497 tokens
6347

NER tags: ['B-Disease', 'I-Disease', 'O']


In [ ]:
train.head()

,tokens,labels
0,"[Identification, of, APC2, ,, a, homologue, of...","[O, O, O, O, O, O, O, O, B-Disease, I-Disease,..."
1,"[The, adenomatous, polyposis, coli, (, APC, ),...","[O, B-Disease, I-Disease, I-Disease, I-Disease..."
2,"[Complex, formation, induces, the, rapid, degr...","[O, O, O, O, O, O, O, O, O]"
3,"[In, colon, carcinoma, cells, ,, loss, of, APC...","[O, B-Disease, I-Disease, O, O, O, O, O, O, O,..."
4,"[Here, ,, we, report, the, identification, and...","[O, O, O, O, O, O, O, O, O, O, O, O, O]"


In [ ]:
i = 9
tokens = X_test[i]
labels = y_test[i]

data = {"token": tokens,"label": labels}
df=pd.DataFrame(data=data)
print(df)

         token      label
0   Occasional          O
1     missense          O
2    mutations          O
3           in          O
4          ATM          O
5         were          O
6         also          O
7        found          O
8           in          O
9       tumour  B-Disease
10         DNA          O
11        from          O
12    patients          O
13        with          O
14           B  B-Disease
15           -  I-Disease
16        cell  I-Disease
17         non  I-Disease
18           -  I-Disease
19    Hodgkins  I-Disease
20   lymphomas  I-Disease
21           (          O
22           B  B-Disease
23           -  I-Disease
24         NHL  I-Disease
25           )          O
26         and          O
27           a          O
28           B  B-Disease
29           -  I-Disease
30         NHL  I-Disease
31        cell          O
32        line          O
33           .          O


In [ ]:
%%time
model = BertTokenClassifier(bert_model='scibert-scivocab-cased',
                            max_seq_length=178,
                            epochs=3,
                            gradient_accumulation_steps=4,
                            learning_rate=3e-5,
                            train_batch_size=16,
                            eval_batch_size=16,
                            validation_fraction=0.,
                            label_list=label_list,
                            ignore_label=['O'])

print(model)

# finetune model on train data
model.fit(X_train, y_train)

# score model on test data
f1_test = model.score(X_test, y_test,'macro')
print("Test f1: %0.02f"%(f1_test))

# get predictions on test data
y_preds = model.predict(X_test)

# print report on classifier stats
print(classification_report(flatten(y_test), flatten(y_preds)))

Building sklearn token classifier...
BertTokenClassifier(bert_model='scibert-scivocab-cased', eval_batch_size=16,
                    gradient_accumulation_steps=4, ignore_label=['O'],
                    label_list=['B-Disease', 'I-Disease', 'O'],
                    learning_rate=3e-05, max_seq_length=178,
                    train_batch_size=16, validation_fraction=0.0)


100%|██████████| 410521600/410521600 [04:19<00:00, 1584858.14B/s]


Loading scibert-scivocab-cased model...


100%|██████████| 410521600/410521600 [04:05<00:00, 1672304.18B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 6347, validation data size: 0



...

Training  :   0%|          | 0/1587 [00:00<?, ?it/s]

Predicting:   0%|          | 0/59 [00:00<?, ?it/s]

Test f1: 90.76


Predicting:   0%|          | 0/59 [00:00<?, ?it/s]

              precision    recall  f1-score   support

   B-Disease       0.89      0.93      0.91       960
   I-Disease       0.88      0.94      0.91      1087
           O       1.00      0.99      0.99     22450

    accuracy                           0.99     24497
   macro avg       0.92      0.95      0.94     24497
weighted avg       0.99      0.99      0.99     24497

CPU times: user 15h 36min 30s, sys: 7min 14s, total: 15h 43min 44s
Wall time: 1h 42min 53s


In [ ]:
import sklearn.metrics as metrics

metrics.confusion_matrix(flatten(y_test), flatten(y_preds))

array([[  889,    37,    34],
       [   18,  1024,    45],
       [   94,   106, 22250]])

In [ ]:
%%bash
DATADIR="JNLPBA"
if test ! -d "$DATADIR";then
    echo "Creating $DATADIR dir"
    mkdir "$DATADIR"
    cd "$DATADIR"
    wget https://raw.githubusercontent.com/cambridgeltl/MTL-Bioinformatics-2016/master/data/JNLPBA/devel.tsv
    wget https://raw.githubusercontent.com/cambridgeltl/MTL-Bioinformatics-2016/master/data/JNLPBA/test.tsv
    wget https://raw.githubusercontent.com/cambridgeltl/MTL-Bioinformatics-2016/master/data/JNLPBA/train.tsv
fi

In [ ]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 10.9 MB/s eta 0:00:00


In [ ]:
! cd transformers/examples/ner/
! wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

/bin/bash: line 1: cd: transformers/examples/ner/: No such file or directory
--2023-10-16 13:36:05--  https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 991 [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]     991  --.-KB/s    in 0s      

2023-10-16 13:36:06 (121 MB/s) - ‘preprocess.py’ saved [991/991]



In [ ]:
%env MAX_LENGTH=128
%env BERT_MODEL=mrm8488/scibert_scivocab-finetuned-CORD19


env: MAX_LENGTH=128
env: BERT_MODEL=mrm8488/scibert_scivocab-finetuned-CORD19
cat: 'JNLPBA/*.raw': No such file or directory


In [ ]:
! python preprocess.py /kaggle/working/JNLPBA/train.txt.tmp $BERT_MODEL $MAX_LENGTH > /kaggle/working/JNLPBA/train.txt
! python preprocess.py /kaggle/working/JNLPBA/test.txt.tmp $BERT_MODEL $MAX_LENGTH > /kaggle/working/JNLPBA/test.txt
! python preprocess.py /kaggle/working/JNLPBA/dev.txt.tmp $BERT_MODEL $MAX_LENGTH > /kaggle/working/JNLPBA/dev.txt

! wget https://raw.githubusercontent.com/allenai/scibert/master/data/ner/JNLPBA/dev.txt -O JNLPBA/dev.raw
! cat JNLPBA/labels.txt
! cat JNLPBA/*.raw | cut -f 4 | sort | grep -v "^$" | uniq > JNLPBA/labels.txt

/bin/bash: line 1: /kaggle/working/JNLPBA/train.txt: No such file or directory
/bin/bash: line 1: /kaggle/working/JNLPBA/test.txt: No such file or directory
/bin/bash: line 1: /kaggle/working/JNLPBA/dev.txt: No such file or directory
--2023-10-16 13:44:58--  https://raw.githubusercontent.com/allenai/scibert/master/data/ner/JNLPBA/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 701448 (685K) [text/plain]
Saving to: ‘JNLPBA/dev.raw’

JNLPBA/dev.raw      100%[===================>] 685.01K  --.-KB/s    in 0.07s   

2023-10-16 13:44:58 (10.1 MB/s) - ‘JNLPBA/dev.raw’ saved [701448/701448]



In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained('allenai/scibert_scivocab_cased')
tokenizer = AutoTokenizer.from_pretrained('mrm8488/scibert_scivocab-finetuned-CORD19')

import torch

with open('JNLPBA/labels.txt', 'r') as f:
    labels = [l.strip() for l in f.readlines()]

label_id_to_label = {i:l for i, l in enumerate(labels)}

def tokenize(sample):
    return tokenizer.encode(' '.join(['CLS', sample, 'SEP']))

def get_prediction(sample):
    tokens = tokenize(sample)
    attention_mask = [1] * len(tokens)
    predictions = model.forward(input_ids=torch.LongTensor([tokens]),
                                attention_mask=torch.LongTensor([attention_mask]))[0].argmax(axis=2).tolist()[0]
    return [label_id_to_label[i] for i in predictions]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sample = 'IL-2 gene expression and NF-kappa B activation through CD28 requires reactive oxygen production by 5-lipoxygenase.'
predictions = get_prediction(sample)
for token, pred in zip(tokenizer.tokenize(sample), predictions):
    print(token, pred)

il B-cell_type
- B-cell_type
2 B-cell_type
gene B-cell_type
expression B-cell_line
and B-cell_type
nf B-cell_type
- B-cell_type
kappa B-cell_type
b B-cell_type
activation B-cell_type
through B-cell_type
cd B-cell_type
##28 B-cell_type
requires B-cell_line
reactive B-cell_type
oxygen B-cell_type
production B-cell_type
by B-cell_line
5 B-cell_line
- B-cell_line
lip B-cell_type
##oxygenase B-cell_line
. B-cell_line
